In [1]:
import numpy as np
import newbridge as nb
import parameters as prm
%matplotlib inline
from matplotlib import pyplot as plt

In [2]:
# load data
import pickle
with open('./data/nem_2D.pkl','rb') as f:
    xout, tout, x_without_noise, euler_param, sim_param = pickle.load(f)

In [3]:
print(xout.shape)
print(tout.shape)

(10, 101, 2)
(10, 101)


In [4]:
print(nb.index_mapping())

{(0, 0): 0, (1, 0): 1, (0, 1): 2, (2, 0): 3, (1, 1): 4, (0, 2): 5, (3, 0): 6, (2, 1): 7, (1, 2): 8, (0, 3): 9}


In [5]:
theta = np.array([1., 1., 2., 2., 4., 4., 5., 5., 6., 6.])
polytheta = nb.hermite_to_ordinary(theta)

[[ 0.39894228  0.          0.         -0.28209479  0.         -0.28209479
   0.          0.          0.          0.        ]
 [ 0.          0.39894228  0.          0.          0.          0.
  -0.48860251  0.         -0.28209479  0.        ]
 [ 0.          0.          0.39894228  0.          0.          0.          0.
  -0.28209479  0.         -0.48860251]
 [ 0.          0.          0.          0.28209479  0.          0.          0.
   0.          0.          0.        ]
 [ 0.          0.          0.          0.          0.39894228  0.          0.
   0.          0.          0.        ]
 [ 0.          0.          0.          0.          0.          0.28209479
   0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.          0.          0.
   0.1628675   0.          0.          0.        ]
 [ 0.          0.          0.          0.          0.          0.          0.
   0.28209479  0.          0.        ]
 [ 0.          0.          0.          0.       

In [6]:
def index_mapping():
    index = 0
    index_map = {}

    for d in range(0, prm.num_hermite_terms):
        for j in range(0, d + 1):
            for i in range(0, d + 1):
                if (i + j == d):
                    index_set = (i, j)
                    index_map[index_set] = index
                    index += 1
    
    return index_map

def hermite_to_ordinary(theta):
    transformation = np.zeros((prm.dof, prm.dof))
    index_map = index_mapping()
    index = 0
    
    mat = np.zeros((prm.num_hermite_terms, prm.num_hermite_terms))
    mat[0, 0] = 0.63161877774606470129
    mat[1, 1] = 0.63161877774606470129
    mat[2, 2] = 0.44662192086900116570
    mat[0, 2] = -mat[2, 2]
    mat[3, 3] = 0.25785728623970555997
    mat[1, 3] = -3 * mat[3, 3]

    for d in range(0, prm.num_hermite_terms):
        for j in range(0, d + 1):
            for i in range(0, d + 1):
                if (i + j == d):
                    if (i >= 2):
                        new_index_set = (i - 2, j)
                        new_index = index_map[new_index_set]
                        transformation[new_index, index] = mat[i - 2, i] * mat[j, j]
                    if (j >= 2):
                        new_index_set = (i, j - 2)
                        new_index = index_map[new_index_set]
                        transformation[new_index, index] = mat[i, i] * mat[j - 2, j]

                    transformation[index, index] = mat[i, i] * mat[j, j]
                    index += 1
                    
    print(transformation)                        
    transformed_theta = np.matmul(transformation, theta)
    return np.transpose(transformed_theta)

In [7]:
mat = index_mapping()
print(mat)
# newtheta = hermite_to_ordinary(theta)

{(0, 0): 0, (1, 0): 1, (0, 1): 2, (2, 0): 3, (1, 1): 4, (0, 2): 5, (3, 0): 6, (2, 1): 7, (1, 2): 8, (0, 3): 9}
